<a href="https://colab.research.google.com/github/alfariziraju/Praktek-Fuzzy-G211220061/blob/main/Praktek_Fuzzy_Sugeno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# --- 1. Domain variabel dan Fungsi Keanggotaan ---

# 1.1 Domain Input
x_suhu = np.arange(0, 101, 1) # Suhu dalam °C (0 sampai 100) [cite: 23]
x_tekanan = np.arange(0, 11, 1) # Tekanan dalam Bar (0 sampai 10) [cite: 24]

# 1.2 Fungsi Keanggotaan (Suhu) - Menggunakan trimf (fungsi segitiga)
# Suhu Rendah [0, 0, 40]
suhu_rendah = fuzz.trimf(x_suhu, [0, 0, 40]) # [cite: 28, 29]
# Suhu Sedang [30, 50, 70]
suhu_sedang = fuzz.trimf(x_suhu, [30, 50, 70]) # [cite: 31, 33]
# Suhu Tinggi [60, 100, 100]
suhu_tinggi = fuzz.trimf(x_suhu, [60, 100, 100]) # [cite: 35, 36]

# 1.3 Fungsi Keanggotaan (Tekanan) - Menggunakan trimf (fungsi segitiga)
# Tekanan Rendah [0, 0, 4]
tekanan_rendah = fuzz.trimf(x_tekanan, [0, 0, 4]) # [cite: 39, 43]
# Tekanan Sedang [3, 5, 7]
tekanan_sedang = fuzz.trimf(x_tekanan, [3, 5, 7]) # [cite: 44, 46]
# Tekanan Tinggi [6, 10, 10]
tekanan_tinggi = fuzz.trimf(x_tekanan, [6, 10, 10]) # [cite: 47, 48]

# --- 2. Fungsi Sugeno ---

def sugeno (suhu_input, tekanan_input):

    # Fuzzifikasi (Menghitung derajat keanggotaan)
    μ_s_rendah = fuzz.interp_membership(x_suhu, suhu_rendah, suhu_input) # [cite: 58]
    μ_s_sedang = fuzz.interp_membership(x_suhu, suhu_sedang, suhu_input)
    μ_s_tinggi = fuzz.interp_membership(x_suhu, suhu_tinggi, suhu_input) # [cite: 60]

    μ_t_rendah = fuzz.interp_membership(x_tekanan, tekanan_rendah, tekanan_input) # [cite: 64]
    μ_t_sedang = fuzz.interp_membership(x_tekanan, tekanan_sedang, tekanan_input)
    μ_t_tinggi = fuzz.interp_membership(x_tekanan, tekanan_tinggi, tekanan_input) # [cite: 65]

    # Aplikasi Aturan (Inferensi)
    # Output Sugeno adalah konstanta (Orde-0)

    # Rule 1: IF suhu rendah AND tekanan rendah -> risiko = 10 [cite: 7, 8]
    w1 = np.fmin(μ_s_rendah, μ_t_rendah) # [cite: 71]
    z1 = 10 # [cite: 73]

    # Rule 2: IF suhu sedang AND tekanan sedang -> risiko = 50 [cite: 9, 10]
    w2 = np.fmin(μ_s_sedang, μ_t_sedang) # [cite: 77]
    z2 = 50 # [cite: 79]

    # Rule 3: IF suhu tinggi OR tekanan tinggi -> risiko = 90 [cite: 11, 12, 85]
    w3 = np.fmax(μ_s_tinggi, μ_t_tinggi) # [cite: 84]
    z3 = 90

    # Rule 4: IF suhu tinggi AND tekanan rendah -> risiko = 60 [cite: 13, 14, 90]
    w4 = np.fmin(μ_s_tinggi, μ_t_rendah) # [cite: 89]
    z4 = 60 # [cite: 92]

    # Defuzzifikasi (Weighted Average / Rata-rata Terbobot Sugeno)
    w = np.array([w1, w2, w3, w4]) # [cite: 97, 98]
    z = np.array([z1, z2, z3, z4]) # [cite: 100, 103]

    # Rumus Defuzzifikasi Sugeno: (Σ(wi * zi)) / (Σwi)
    result = np.sum(w * z) / np.sum(w) # [cite: 106]

    return result # [cite: 109]

# --- 3. Widget Interaktif dan Visualisasi ---

def interactive_sugeno (suhu, tekanan): # [cite: 113]
    """Fungsi untuk menghitung Sugeno dan menampilkan hasil serta plot."""

    hasil = sugeno(suhu, tekanan) # [cite: 114]

    print(f"🔥 Hasil Prediksi Risiko (Sugeno): {hasil:.2f}") # [cite: 117]

    # Visualisasi # [cite: 120]
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4)) # [cite: 122]

    # Plot Suhu # [cite: 125, 131]
    ax1.plot(x_suhu, suhu_rendah, label="Rendah")
    ax1.plot(x_suhu, suhu_sedang, label="Sedang")
    ax1.plot(x_suhu, suhu_tinggi, label="Tinggi")
    ax1.axvline(suhu, color='red', linestyle='--') # Garis input Suhu # [cite: 135]
    ax1.set_title("Fungsi Keanggotaan Suhu") # [cite: 136]
    ax1.legend() # [cite: 137]

    # Plot Tekanan # [cite: 138]
    ax2.plot(x_tekanan, tekanan_rendah, label="Rendah")
    ax2.plot(x_tekanan, tekanan_sedang, label="Sedang")
    ax2.plot(x_tekanan, tekanan_tinggi, label="Tinggi")
    ax2.axvline(tekanan, color='red', linestyle='--') # Garis input Tekanan # [cite: 139]
    ax2.set_title("Fungsi Keanggotaan Tekanan") # [cite: 139]
    ax2.legend() # [cite: 151]

    plt.show() # [cite: 150]

# Jalankan slider interaktif # [cite: 154]
interact(
    interactive_sugeno,
    # Slider Suhu (min=0, max=100, step=1, default=50) # [cite: 161]
    suhu=FloatSlider(min=0, max=100, step=1, value=50),
    # Slider Tekanan (min=0, max=10, step=0.5, default=5) # [cite: 161]
    tekanan=FloatSlider(min=0, max=10, step=0.5, value=5)
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 10.9 MB/s eta 0:00:00


interactive(children=(FloatSlider(value=50.0, description='suhu', step=1.0), FloatSlider(value=5.0, descriptio…

<function __main__.interactive_sugeno(suhu, tekanan)>